In [1]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path('/Users/rly/Documents/NWB/spyglass-workspace') # change this to your desired directory
# base_dir = Path('/hdd/dj') # change this to your desired directory
raw_dir = base_dir / 'raw'
analysis_dir = base_dir / 'analysis'
recording_dir = base_dir / 'recording'
sorting_dir = base_dir / 'sorting'
waveforms_dir = base_dir / 'waveforms'
tmp_dir = base_dir / 'tmp'
kachery_cloud_dir = base_dir / '.kachery_cloud'

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)
os.makedirs(recording_dir, exist_ok=True)
os.makedirs(sorting_dir, exist_ok=True)
os.makedirs(waveforms_dir, exist_ok=True)
os.makedirs(tmp_dir, exist_ok=True)
os.makedirs(kachery_cloud_dir, exist_ok=True)

# set dj config
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['database.port'] = 3306
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage': str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}
dj.config["enable_python_native_blobs"] = True

# set env vars
os.environ['SPYGLASS_BASE_DIR'] = str(base_dir)
os.environ['SPYGLASS_RECORDING_DIR'] = str(recording_dir)
os.environ['SPYGLASS_SORTING_DIR'] = str(sorting_dir)
os.environ['SPYGLASS_WAVEFORMS_DIR'] = str(waveforms_dir)
os.environ['SPYGLASS_TEMP_DIR'] = str(tmp_dir)
os.environ['KACHERY_CLOUD_DIR'] = str(kachery_cloud_dir)
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

import spyglass as sg
import spyglass.common as sgc
import spyglass.data_import as sgdi
import spyglass.spikesorting as sgs
import warnings
warnings.simplefilter("always")

[2023-02-07 10:13:26,829][INFO]: Connecting root@localhost:3306
[2023-02-07 10:13:26,980][INFO]: Connected root@localhost:3306
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/position_tools/core.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'SpikeGadgets'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'TDT_Rig1'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'TDT_Rig2'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'PCS'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'RCS'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'RNS'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'NeuroOmega'} using insert1.
Populate: Populating table DataAcquisitionDeviceSystem with data {'data_acquisition_device_system': 'Neuralynx'} using i

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# sgc.DataAcquisitionDeviceSystem.insert1(
#     {"data_acquisition_device_system": "Neuropixels Acquisition System"}, 
#     skip_duplicates=True
# )

In [3]:
# sgc.ProbeType.insert1({
#   "probe_type": "128c-4s6mm6cm-15um-26um-sl",
#   "probe_description": "A Livermore flexible probe with 128 channels, 4 shanks, 6 mm shank length, 6 cm ribbon length. 15 um contact diameter, 26 um center-to-center distance (pitch), single-line configuration.",
#   "manufacturer": "Lawrence Livermore National Lab",
#   "num_shanks": 4,
# }, skip_duplicates=True)
# sgc.ProbeType()

In [4]:
# Define the name of the file that you copied and renamed; make sure it's something unique. 
# nwb_file_name = 'sub-CuffLinks_ses-CuffLinks-LT03-g3_behavior+ecephys.nwb'
nwb_file_name = 'sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb'

filename, file_extension = os.path.splitext(nwb_file_name)
# This is a copy of the original nwb file, except it doesn't contain the raw data (for storage reasons)
nwb_file_name2 = filename + '_' + file_extension

In [7]:
sgdi.insert_sessions(nwb_file_name)

Creating a copy of NWB file sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb with link to raw ephys data: sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb


/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/rly/Documents/NWB/spyglass-workspace/raw/sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/datajoint/external.py:276: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:


Populate Session...
Institution...
Lab...
LabMember...
Please add the Google user ID for Emily A. Aery Jones in the LabMember.LabMemberInfo table if you want to give them permission to manually curate sortings by this team.
Subject...
Populate DataAcquisitionDevice...
No conforming data acquisition device metadata found.

Populate CameraDevice...
No conforming camera device metadata found.

Populate Probe...
No conforming probe metadata found.

Skipping Apparatus for now...
IntervalList...
No epochs found in NWB file.
DataAcquisitionDevice with name New Device does not exist. Cannot link Session with DataAcquisitionDevice in Session.DataAcquisitionDevice.
Populate ElectrodeGroup...
Populate Electrode...
Populate Raw...
Importing raw data: Sampling rate:	30000.0 Hz
Number of valid intervals:	1
Populate SampleCount...
Unable to import SampleCount: no data interface named "sample_count" found in sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb.
Populate DIOEvents...
No confor

In [8]:
sgc.Probe.create_from_nwbfile(
    nwb_file_name=nwb_file_name2,
    nwb_device_name="Device",
    probe_id="Neuropixels 1.0 Giocomo Lab Configuration",
    probe_type="Neuropixels 1.0",
    contact_side_numbering=True
)

In [9]:
sgc.Probe()

probe_id a unique ID for this probe and dynamic configuration,probe_type,data_acquisition_device_name,"contact_side_numbering if True, then electrode contacts are facing you when numbering them"
My Custom Probe Configuration 1,My Custom Probe,None,True
Neuropixels 1.0 Giocomo Lab Configuration,Neuropixels 1.0,None,True


In [10]:
sgc.Electrode()

nwb_file_name name of the NWB file,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,probe_id a unique ID for this probe and dynamic configuration,probe_shank shank number within probe. should be unique within a Probe,probe_electrode electrode ID that is output from the data acquisition system,region_id,name unique label for each contact,original_reference_electrode the configured reference electrode for this electrode,x the x coordinate of the electrode position in the brain,y the y coordinate of the electrode position in the brain,z the z coordinate of the electrode position in the brain,filtering description of the signal filtering,impedance electrode impedance,"bad_channel if electrode is ""good"" or ""bad"" as observed during recording",x_warped x coordinate of electrode position warped to common template brain,y_warped y coordinate of electrode position warped to common template brain,z_warped z coordinate of electrode position warped to common template brain,contacts label of electrode contacts used for a bipolar signal - current workaround
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,0,None,None,None,21,0,-1,987.0,431.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,1,None,None,None,21,1,-1,987.0,431.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,2,None,None,None,21,2,-1,987.0,429.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,3,None,None,None,21,3,-1,987.0,429.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,4,None,None,None,21,4,-1,987.0,427.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,5,None,None,None,21,5,-1,987.0,427.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,6,None,None,None,21,6,-1,987.0,425.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,7,None,None,None,21,7,-1,987.0,425.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,8,None,None,None,21,8,-1,987.0,423.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,9,None,None,None,21,9,-1,987.0,423.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,


In [11]:
sgc.Electrode.create_from_config(nwb_file_name2)

Updated Electrode with ID 0.
Updated Electrode with ID 1.
Updated Electrode with ID 2.
Updated Electrode with ID 3.
Updated Electrode with ID 4.
Updated Electrode with ID 5.
Updated Electrode with ID 6.
Updated Electrode with ID 7.
Updated Electrode with ID 8.
Updated Electrode with ID 9.
Updated Electrode with ID 10.
Updated Electrode with ID 11.
Updated Electrode with ID 12.
Updated Electrode with ID 13.
Updated Electrode with ID 14.
Updated Electrode with ID 15.
Updated Electrode with ID 16.
Updated Electrode with ID 17.
Updated Electrode with ID 18.
Updated Electrode with ID 19.
Updated Electrode with ID 20.
Updated Electrode with ID 21.
Updated Electrode with ID 22.
Updated Electrode with ID 23.
Updated Electrode with ID 24.
Updated Electrode with ID 25.
Updated Electrode with ID 26.
Updated Electrode with ID 27.
Updated Electrode with ID 28.
Updated Electrode with ID 29.
Updated Electrode with ID 30.
Updated Electrode with ID 31.
Updated Electrode with ID 32.
Updated Electrode wi

In [12]:
sgc.Electrode()

nwb_file_name name of the NWB file,electrode_group_name electrode group name from NWBFile,electrode_id the unique number for this electrode,probe_id a unique ID for this probe and dynamic configuration,probe_shank shank number within probe. should be unique within a Probe,probe_electrode electrode ID that is output from the data acquisition system,region_id,name unique label for each contact,original_reference_electrode the configured reference electrode for this electrode,x the x coordinate of the electrode position in the brain,y the y coordinate of the electrode position in the brain,z the z coordinate of the electrode position in the brain,filtering description of the signal filtering,impedance electrode impedance,"bad_channel if electrode is ""good"" or ""bad"" as observed during recording",x_warped x coordinate of electrode position warped to common template brain,y_warped y coordinate of electrode position warped to common template brain,z_warped z coordinate of electrode position warped to common template brain,contacts label of electrode contacts used for a bipolar signal - current workaround
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,0,Neuropixels 1.0 Giocomo Lab Configuration,0,0,21,0,-1,987.0,431.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,1,Neuropixels 1.0 Giocomo Lab Configuration,0,1,21,1,-1,987.0,431.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,2,Neuropixels 1.0 Giocomo Lab Configuration,0,2,21,2,-1,987.0,429.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,3,Neuropixels 1.0 Giocomo Lab Configuration,0,3,21,3,-1,987.0,429.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,4,Neuropixels 1.0 Giocomo Lab Configuration,0,4,21,4,-1,987.0,427.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,5,Neuropixels 1.0 Giocomo Lab Configuration,0,5,21,5,-1,987.0,427.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,6,Neuropixels 1.0 Giocomo Lab Configuration,0,6,21,6,-1,987.0,425.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,7,Neuropixels 1.0 Giocomo Lab Configuration,0,7,21,7,-1,987.0,425.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,8,Neuropixels 1.0 Giocomo Lab Configuration,0,8,21,8,-1,987.0,423.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,shank0,9,Neuropixels 1.0 Giocomo Lab Configuration,0,9,21,9,-1,987.0,423.0,280.0,Neuropixels default filters,nan,False,0.0,0.0,0.0,


In [ ]:
# generate the yaml for mapping the Electrodes table to existing Probe.Electrode entries
import yaml

electrode_dicts = []
for i in range(384):
    d = {
        "electrode_id": i,
        "probe_id": "Neuropixels 1.0 Giocomo Lab Configuration",
        "probe_shank": 0,
        "probe_electrode": i,
        "bad_channel": "False",
    }
    electrode_dicts.append(d)

with open('test.yml', 'w') as outfile:
    yaml.dump(electrode_dicts, outfile, default_flow_style=False, sort_keys=False)

In [ ]:
sgc.Session().delete(safemode=False, force_parts=True)
sgc.DataAcquisitionDeviceSystem().delete(safemode=False)
sgc.DataAcquisitionDeviceAmplifier().delete(safemode=False)
sgc.DataAcquisitionDevice().delete(safemode=False)
sgc.Institution().delete(safemode=False, force_parts=True)
sgc.Lab().delete(safemode=False)
sgc.LabTeam().delete(safemode=False)
sgc.LabMember().delete(safemode=False)
sgc.Subject().delete(safemode=False)
sgc.Probe().delete(safemode=False)
sgc.ProbeType().delete(safemode=False)
sgc.Nwbfile().delete(safemode=False)
sgc.Nwbfile().cleanup(delete_files=True)
sgs.SortGroup().delete(safemode=False)
sgs.SortInterval().delete(safemode=False)
sgs.SpikeSorterParameters().delete(safemode=False)
sgs.SpikeSortingPreprocessingParameters().delete(safemode=False)
sgs.SpikeSortingRecordingSelection().delete(safemode=False)
sgs.Curation().delete(safemode=False)
sgc.BrainRegion().delete(safemode=False)